PPCA lavora con più variabili osservabili anzichè una sola

In [1]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import scipy

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
scaler = StandardScaler()

In [3]:
data = np.load('prova.npy', allow_pickle=True).item()

## test with new data

In [4]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets
import deepemogp.signal.behavior as behavior

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np

show = False
# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(2, 1))

# definition of the physiological signals to be extracted
eda = physio.EDA(f3)
hr = physio.HR(f3)

pupil = behavior.PUPIL(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr,pupil}, subjects={'2'})

for s in d.signals:
    # preprocess ...
    #s.preprocess(show=show)

    # ... and extract features from each signal type
    s.feature_ext.extract_feat_without_preprocess(s, show=show)

hr_data = list(d.signals)[0].features[:100]
pupil_data = list(d.signals)[1].features[:100]

print(pupil_data)
print()
print(hr_data)

>> Loading HR for subject 2 and all sessions from dataset FEAR
>> Loading PUPIL for subject 2 and all sessions from dataset FEAR
>> Extracting mean features from HR signal, adopting (2, 1) window ...
>> Extracting mean features from PUPIL signal, adopting (2, 1) window ...
          0
0  2.176351
1  2.134074
2  2.159014
3  2.186318

          0
0 -0.002085
1 -0.002064
2 -0.002355
3 -0.005520
4 -0.002045


## end test

prendo un'implementazione di sppca e la applico al nostro caso

different types of observable data

1) gaze
2) fisio
    2.1) heart rate variabilty
    2.2) eda phasic value
3) social anxiety
4) aspettativa del dolore

gaze =

In [5]:
gaze = pupil_data
hr = hr_data
#eda = data['eda']
'''social_anx = data['social anxiety'][:,np.newaxis]
pain_exp = data['pain expectation'][:,np.newaxis]'''


"social_anx = data['social anxiety'][:,np.newaxis]\npain_exp = data['pain expectation'][:,np.newaxis]"

In [6]:
print(gaze.shape)
print(hr.shape)

(4, 1)
(5, 1)


In [7]:
N_gaze = gaze.shape[0]
D_gaze = gaze.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

#N_eda = eda.shape[0]
#D_eda = eda.shape[1]
K = 1

print(gaze.T)
print()
print(hr.T)

          0         1         2         3
0  2.176351  2.134074  2.159014  2.186318

          0         1         2        3         4
0 -0.002085 -0.002064 -0.002355 -0.00552 -0.002045


In [8]:
with pm.Model() as sPPCA:
    #dati osservabili
    gaze_data = pm.MutableData("gaze_data", gaze.T)
    hr_data = pm.MutableData("hr_data", hr.T)
    #eda_data = pm.MutableData("eda_data", eda.T)

    #matrici pesi
    Wgaze = pm.Normal('Wgaze', mu=at.zeros([D_gaze, K]), sigma=2.0 * at.ones([D_gaze, K]), shape=[D_gaze, K])
    Whr = pm.Normal('Whr', mu=at.zeros([D_hr, K]), sigma=2.0 * at.ones([D_hr, K]), shape=[D_hr, K])
    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([N_gaze, K]), sigma=at.ones([N_gaze, K]), shape=[N_gaze, K])

    x_gaze = pm.Normal('x_gaze', mu=Wgaze.dot(c.T), sigma=at.ones([D_gaze, N_gaze]), shape=[D_gaze, N_gaze], observed=gaze_data)
    x_hr = pm.Normal('x_hr', mu=Whr.dot(c.T), sigma=at.ones([D_hr,N_hr]) ,shape=[D_hr, N_hr], observed=hr_data)
    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

In [9]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

'.gv.pdf'

In [10]:
sPPCA.free_RVs

[Wgaze ~ N(f(), f()), Whr ~ N(f(), f()), c ~ N(f(), f())]

In [11]:
with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,progressbar=True)

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

ValueError: Input dimension mismatch. One other input has shape[1] = 5, but input[1].shape[1] = 4.
Apply node that caused the error: Elemwise{sub,no_inplace}(hr_data, ScipyGer{non-destructive}.0)
Toposort index: 61
Inputs types: [TensorType(float64, (None, None)), TensorType(float64, (1, None))]
Inputs shapes: [(1, 5), (1, 4)]
Inputs strides: [(40, 8), (32, 8)]
Inputs values: [array([[-0.0020846 , -0.00206435, -0.002355  , -0.00551986, -0.00204512]]), array([[-0.05898346,  0.40610175, -0.87491646,  1.05785567]])]
Outputs clients: [[SpecifyShape(Elemwise{sub,no_inplace}.0, NoneConst, TensorConstant{5})]]

Backtrace when the node is created (use Aesara flag traceback__limit=N to make it longer):
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/pymc/variational/opvi.py", line 1230, in _single_symbolic_varlogp_and_datalogp
    varlogp, datalogp = self.symbolic_single_sample([self.model.varlogp, self.model.datalogp])
  File "/home/paolo/matteo/matt

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"gaze_data": gaze})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
hr_pred = posterior_predictive.posterior_predictive["x_hr"]

In [ ]:
gaze

In [ ]:
print(hr_pred[0])

In [ ]:
print(hr_pred[0].shape)

In [ ]:
print(float(np.mean(hr_pred[0])))

median abs deviation prb is not the optimal solution

In [ ]:
hr_pred_mean = (stats.median_abs_deviation(hr_pred[0]))[0]
hr_pred_mean

In [ ]:
hr[0]

In [ ]:
#hr_pred_mode= pd.Series([0.006,0.01,0.02,0.003])

hr and hr_pred_mode devono avere same shape

In [ ]:
train_accuracy_hr = (np.subtract(hr[0], hr_pred_mean))
print('\n\t avg distance hr: \n' + str(train_accuracy_hr))
#print(' ')

In [ ]:
with sPPCA:
    # update values of predictors:
    pm.set_data({"hr_data": hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)
    gaze_pred = posterior_predictive.posterior_predictive["x_gaze"]

In [ ]:
gaze_pred

In [ ]:
gaze_pred_mean = np.mean(gaze_pred, axis=0).T.reshape(-1)
gaze_true = gaze.reshape(-1)
reconstructed_r2 = r2_score(gaze_true, gaze_pred_mean)

print('\n\tReconstructed X R2 score: ' + str(reconstructed_r2))

In [ ]:
import deepemogp.feature_extractor as feature_extractor

In [ ]:
f = feature_extractor.FE('wavelet', window=(8,6), params={'w_mother':'db3','w_maxlev':1})

In [ ]:
x = [3, 7, 1, 1, -2, 5, 4, 6]

In [ ]:
f.apply(x)